# Processamento do PGN

In [ ]:
!pip install neptune-client

     |████████████████████████████████| 276 kB 5.3 MB/s 
     |████████████████████████████████| 829 kB 35.1 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 180 kB 62.4 MB/s 
     |████████████████████████████████| 131 kB 63.2 MB/s 
     |████████████████████████████████| 79 kB 7.2 MB/s 
     |████████████████████████████████| 8.4 MB 35.3 MB/s 
     |████████████████████████████████| 138 kB 56.9 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 127 kB 58.3 MB/s 
     |████████████████████████████████| 130 kB 49.8 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.13.3-py2.py3-none-any.whl size=481965 sha256=165f6913081839e7a59bea820f99bd562a750e16a6f50c93724e6d73434b2dff
  Stored in directory: /root/.cache/pip/wheels/0f/57/12/8fae2c4d320108451ca55c0c1288b69721f496062ed438d63d
  Created wheel for future: filena

In [ ]:
import neptune.new as neptune

run = neptune.init(
    project="heldervj/chess-nlp",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI0OTI1MzNlYS0xNTY3LTQwZjctYjMxNC05ZDNmNGFkZTViMTEifQ==",
)  # your credentials

Info (NVML): Driver Not Loaded. GPU usage metrics may not be reported. For more information, see https://docs-legacy.neptune.ai/logging-and-managing-experiment-results/logging-experiment-data.html#hardware-consumption 


https://app.neptune.ai/heldervj/chess-nlp/e/CHES-30
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# In order to reproduce the experiment, change this path to the file download from:
# https://zenodo.org/record/5767528#.YbGYjvHMLDI

file = open('drive/MyDrive/chess_dataset/chess_games.txt', 'rb')
file = file.read()

In [ ]:
print(file[:100])

b'[Event "12th Forchheim Open A 2013"]\r\n[Site "Forchheim GER"]\r\n[Date "31.12.29"]\r\n[Round "3.1"]\r\n[Whi'


-------------------

In [ ]:
import pandas as pd

In [ ]:
df_info = pd.DataFrame(columns=[
                                'id_partida',
                                'WhiteElo',
                                'BlackElo',
                                'NbMoves',
                                'EventType',
                                'Result'])

df_pgn = pd.DataFrame(columns=[
                               'id_partida',
                               'pgn'])

In [ ]:
rows_to_process = 3_500_000_000

len(file)

9226295224

In [ ]:
from tqdm import tqdm
import gc

id_partida = 0
in_game_detail = False
in_moves = False

whiteElo = None
blackElo = None
nbMoves = 0
eventType = None
result = None

game = b''

counter = 1

for line in tqdm(file[:rows_to_process].splitlines()[12297919:]):
    if b'Event' in line:
        in_game_detail = True
    
    if in_game_detail:
        if b'WhiteElo' in line:
            try:
                whiteElo = line.split(b'"')[1].decode()
            except:
                whiteElo = ''

        elif b'BlackElo' in line:
            try:
                blackElo = line.split(b'"')[1].decode()
            except:
                blackElo = ''

        elif b'PlyCount' in line:
            try:
                nbMoves = line.split(b'"')[1].decode()
            except:
                nbMoves = ''

        elif b'EventType' in line:
            try:
                eventType = line.split(b'"')[1].decode()
            except:
                eventType = ''

        elif b'Result' in line:
            try:
                result = line.split(b'"')[1].decode()
            except:
                result = ''

        elif b'' == line:
            in_game_detail = False
            in_moves = True

            df_info.loc[id_partida] = [id_partida,
                                    whiteElo,
                                    blackElo,
                                    nbMoves,
                                    eventType,
                                    result]

            whiteElo = None
            blackElo = None
            nbMoves = 0
            eventType = None
            result = None

            


    elif in_moves:
        if b'' != line:
            game = game + b' ' + line
        else:
            df_pgn.loc[id_partida] = [id_partida,
                                      game]

            id_partida += 1

            run['preproc/parsing'].log(id_partida)

            game = b''

            in_moves = False

            if id_partida % 6000 == 0:
                df_info.to_csv(f'drive/MyDrive/chess_dataset/csv_files/info_{counter}.csv', index=False)
                df_pgn.to_csv(f'drive/MyDrive/chess_dataset/csv_files/pgn_{counter}.csv', index=False)

                df_info = df_info[0:0]
                df_pgn = df_pgn[0:0]

                gc.collect()

                counter += 1
    

df_info.to_csv(f'drive/MyDrive/chess_dataset/csv_files/info_{counter}.csv', index=False)
df_pgn.to_csv(f'drive/MyDrive/chess_dataset/csv_files/pgn_{counter}.csv', index=False)

print(df_info.shape)
print(df_pgn.shape)

In [ ]:
df_info.to_csv('drive/MyDrive/chess_dataset/info.csv', index=False)
df_pgn.to_csv('drive/MyDrive/chess_dataset/pgn.csv', index=False)

print(df_info.shape)
print(df_pgn.shape)

(358729, 6)
(358728, 2)
